In [14]:
import pandas as pd
import csv
import glob
from datetime import datetime


CSV_GLOB_PATTERN = '../plays/*.csv'
FROM = '2018-01-01'
TO = None


POSSIBLE_TIMESTAMP_COLUMN_NAMES = ['uts', 'timestamp']
POSSIBLE_ARTIST_COLUMN_NAMES = ['artist']
POSSIBLE_TITLE_COLUMN_NAMES = ['title', 'track']
POSSIBLE_ALBUM_COLUMN_NAMES = ['album']


PLAYED_AT = 'played_at_utc'
ARTIST_NAME = 'artist_name'
TRACK_TITLE = 'track_title'
ALBUM_TITLE = 'album_title'
DF_HEADER = [PLAYED_AT, ARTIST_NAME, TRACK_TITLE, ALBUM_TITLE]


TOP_N = 25


def get_column_name(df, possible_column_names):
    for n in possible_column_names:
        if n in df:
            return n

        
def timestamp_to_datetime(timestamp):
    return datetime.fromtimestamp(int(timestamp))


def strip_or_set_none(text):
    return text.strip() if text else None


def load_data_frame():
    result = pd.DataFrame(columns=DF_HEADER)
    
    for file_path in glob.glob(CSV_GLOB_PATTERN):
        rows = None
        with open(file_path, 'r') as csv_in:
            rows = list(csv.DictReader(csv_in))
        df = pd.DataFrame(rows)

        timestamp_column_name = get_column_name(df, POSSIBLE_TIMESTAMP_COLUMN_NAMES)
        artist_column_name = get_column_name(df, POSSIBLE_ARTIST_COLUMN_NAMES)
        title_column_name = get_column_name(df, POSSIBLE_TITLE_COLUMN_NAMES)
        album_column_name = get_column_name(df, POSSIBLE_ALBUM_COLUMN_NAMES)
        
        df.rename(columns={timestamp_column_name: PLAYED_AT,
                           artist_column_name: ARTIST_NAME,
                           title_column_name: TRACK_TITLE,
                           album_column_name: ALBUM_TITLE}, inplace=True)
        
        result = result.append(df[DF_HEADER])
    result[PLAYED_AT] = result[PLAYED_AT].apply(timestamp_to_datetime)
    result[ALBUM_TITLE] = result[ALBUM_TITLE].apply(strip_or_set_none)
    result[TRACK_TITLE] = result[TRACK_TITLE].apply(strip_or_set_none)
    result[ARTIST_NAME] = result[ARTIST_NAME].apply(strip_or_set_none)
    return result

In [15]:
df = load_data_frame()

In [18]:
FROM = FROM if FROM else '1970-01-01'
TO = TO if TO else datetime.now().strftime('%Y-%m-%d')
df = df[(df[PLAYED_AT] >= FROM) & (df[PLAYED_AT] <= TO)]    

In [19]:
df\
    .groupby([TRACK_TITLE, ARTIST_NAME])[TRACK_TITLE]\
    .count()\
    .reset_index(name='count').sort_values(['count'], ascending=False)\
    .head(TOP_N)

,track_title,artist_name,count
1805,Finesse (Remix) [feat. Cardi B],Bruno Mars,60
1471,Dragon Fish,C418,50
1860,Forbidden Lovers,King Garbage,42
4155,Peak,Drake,42
6425,morten vs gojira,morten,41
2997,Knew U,The Pharcyde,41
2639,Ich weiss,Said,41
5245,Switch,6LACK,40
72,24K Magic,Bruno Mars,39
3209,Lieblingsmensch,Remoe,38


In [20]:
df\
    .query('{}!=None'.format(ALBUM_TITLE))\
    .groupby([ALBUM_TITLE, ARTIST_NAME])[ALBUM_TITLE]\
    .count()\
    .reset_index(name='count').sort_values(['count'], ascending=False)\
    .head(TOP_N)

,album_title,artist_name,count
53,1997,Alicks,419
3523,tru. (Deluxe Edition),Cro,289
1872,Make It Sweat,King Garbage,264
2550,Scorpion,Drake,171
2020,My Krazy Life (Deluxe),YG,152
2092,No News Is Good News,Phonte,144
1939,Minecraft - Volume Alpha,C418,139
29,10551 moabit island season 1,morten,130
2176,One and Only,Sero,118
2832,Superorganism,Superorganism,113


In [21]:
df.groupby([ARTIST_NAME])[ARTIST_NAME]\
    .count()\
    .reset_index(name='count').sort_values(['count'], ascending=False)\
    .head(TOP_N)

,artist_name,count
64,Alicks,440
532,Drake,354
381,Cro,309
1038,King Garbage,264
2088,YG,248
1889,The Internet,215
1001,Kanye West,201
2147,morten,198
343,Christian Scott aTunde Adjuah,198
276,C418,189
